In [ ]:
import numpy as np
from IPython.display import Image
from IPython.display import display
import numpy as np
import pandas as pd
from scipy.sparse import load_npz
from tqdm import tqdm
from google.colab import drive
from google.colab import files
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.utils import shuffle 
from keras.models import Model
from keras.layers import Input,Embedding, Flatten, Dense, Concatenate
from keras.layers import Activation
from keras.optimizers import SGD,Adam

# Load data

In [ ]:
drive.mount('/content/drive')
test_csr=load_npz('/content/drive/MyDrive/recom/train_csr.npz')
print(test_csr.shape)

# Create dataframe

In [ ]:
rows, cols = test_csr.nonzero()
df2 = pd.DataFrame(columns=[ 'user_id', 'movie_id','rating'])
df2['user_id'] = cols
df2['movie_id'] = rows
df2['rating'] = test_csr.data
df2

# split data

In [ ]:
N=df2.user_id.max()+1
M=df2.movie_id.max()+1
# df3=df2.loc[1:1000000,:]
df3=shuffle(df2)
cutoff=int(0.8*len(df3))
df_train=df3.iloc[:cutoff]
df_test=df3.iloc[cutoff:]

# Hyper parameters

In [ ]:
K=10  
epochs=50
batch_size=64
learning_rate=0.01
momentum =0.9

# model

In [ ]:
u=Input(shape=(1,))
m=Input(shape=(1,))
u_embedding=Embedding(N,K)(u)
m_embedding=Embedding(M,K)(m)
u_embedding=Flatten()(u_embedding)
m_embedding=Flatten()(m_embedding)
X=Concatenate()([u_embedding,m_embedding])

X=Dense(400)(X)
X=Activation('relu')(X)
X=Dense(1, activation='sigmoid')(X)
model=Model(inputs=(u,m),outputs=(X))
model.compile(loss='mse', optimizer=SGD(learning_rate=learning_rate,momentum =momentum),metrics=['mse'])

model.summary()

# Train model

In [ ]:
recom = model.fit(
    x=[df_train.user_id.values,df_train.movie_id.values], 
    y=df_train.rating.values,
    epochs=epochs, 
    batch_size=batch_size,
    validation_data=([df_test.user_id.values,df_test.movie_id.values],
    df_test.rating.values ))

# Learning curve

In [ ]:
plt.plot(recom.history['loss'],label="train loss")
plt.plot(recom.history['val_loss'],label="test loss")
plt.legend()
plt.show()

In [ ]:
prediction=model.predict([df_test.user_id.values,df_test.movie_id.values])